In [2]:
os.environ['KSC_PATH'] = '../../build/bin/ksc'

In [3]:

import torch
import torch.nn as nn
import torch.nn.functional as F

In [60]:
@torch.jit.script
def bar(x : torch.Tensor):
    if torch.mean(x) < 0:
        t = -0.125*x
    else:
        t = 1/2 * x ** 2
    return torch.sin(t)*t

x0 = torch.tensor([2.2, -1.1])
bar(x0)


tensor([1.5986, 0.3441])

In [27]:
print(bar.code)

def bar(x: Tensor) -> Tensor:
  if bool(torch.lt(x, 0)):
    t = torch.mul(x, -0.125)
  else:
    t = torch.mul(torch.pow(x, 2), 0.5)
  return torch.mul(torch.sin(t), t)



In [64]:
x = torch.tensor(x0, requires_grad=True)
y = bar(x)
# y.backward(torch.tensor(1.0))

torch.autograd.grad(y, x)

None


In [23]:
dir(bar)

['__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'code',
 'get_debug_state',
 'graph',
 'graph_for',
 'inlined_graph',
 'name',
 'qualified_name',
 'save',
 'save_to_buffer',
 'schema']

In [6]:
from ksc.ts2k import ts2mod
from ksc.type import Type

ks_str = """
(def f Float (x : Float) (mul x 2.0))
"""

utils.generate_and_compile_cpp_from_ks(ks_str, "f", [Type.Float])



ModuleNotFoundError: No module named 'ksc.ts2k'

In [37]:
import pybind11

ModuleNotFoundError: No module named 'pybind11'